# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

# Data Fetching

In [3]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16],low_memory=False)
    out= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19],low_memory=False)
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# Principal component analysis Decomposition

In [4]:
from sklearn.decomposition import PCA
import warnings
scaler_obj1=PCA()
scaler_obj2=PCA()
X1=scaler_obj1.fit_transform(A1)
Y1=scaler_obj2.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

# Parameter Tuning

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [7,8,10,],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'n_estimators' : [80,90,100,200,400,1000],
        'objective': ['reg:squarederror','count:poisson']
    }

    xgb_model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)

    gsearch = RandomizedSearchCV(estimator = xgb_model, param_distributions = param_tuning, n_iter = 5, cv = 2, verbose=2, random_state=0, n_jobs = -1)

    
    grid_result = MultiOutputRegressor(gsearch).fit(x_train, y_train)

    return grid_result.estimators_[0].best_params_

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=0)

params=hyperParameterTuning(x_train,y_train)
params

# Model

In [6]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(XGBRegressor(tree_method='gpu_hist', gpu_id=0,max_depth=8,cosample_bytree=.5,learning_rate=.1,min_child_weight=3,
                   n_estimators=200,subsample=.7))

#training the model
model_fit1=model1.fit(x_train, y_train)
print("Model training is Done!!")

#saving as file
filename1 = 'xgboost.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

[13:21:24] WARNING: ../src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




KeyboardInterrupt: 

# Error Analysis

In [118]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')

r2 score on train data 0.9315455074103038
r2 score on test data 0.9310061554157045
Mean Absolute Error: 0.0008703509
Mean Squared Error: 9.53231e-06
Root Mean Squared Error: 0.003087444
 



# Prediction of particles

In [10]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("xgboost.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

/home/utkarsh/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[11:02:14] WARNING: ../src/gbm/gbtree.cc:386: Loading from a raw memory buffer on CPU only machine.  Changing tree_method to hist.
[11:02:14] WARNING: ../src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[11:02:14] WARNING: ../src/gbm/gbtree.cc:386: Loading from a raw memory buffer on CPU only machine.  Changing tree_method to hist.
[11:02:14] WARNING: ../src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[11:02:14] WARNING: ../src/gbm/gbtree.cc:386: Loading from a raw memory buffer on CPU only machine.  Changing tree_method to hist.
[11:02:14] WARNING: ../src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[11:02:14] WARNING: ../src/gbm/gbtree.cc:386: Loading from a raw memory buffer on CPU only machine.  Changing tree_method to hist.
[11:02:14] WARNING: ../src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[11:02:14] WARNING: ../src/gbm/gbtree.cc:386: Loading from a raw memory buffer on CPU only machine.  Changing tr

In [11]:
x_test=scaler_obj1.inverse_transform(x_test)
y_test_pred=scaler_obj2.inverse_transform(y_test_pred)
y_test=scaler_obj2.inverse_transform(y_test)
pd.DataFrame(y_test_pred)

,0,1,2,3,4,5,6
0,47.849255,11.8641,-80.664482,490.139801,2.968223,3.418961,3.777236
1,31.278259,11.8641,-5.546434,341.499512,10.155688,10.203463,13.379510
2,51.198467,11.8641,-25.094217,-132.613449,13.112359,15.961463,18.418491
3,49.207836,11.8641,-44.744858,681.868164,29.937065,35.181923,34.957455
4,33.853252,11.8641,-5.958789,-82.559654,6.641768,8.177238,11.174068
...,...,...,...,...,...,...,...
432566,34.035519,11.8641,4.500061,-2.882016,6.573976,8.000960,9.871785
432567,50.424248,11.8641,-70.336044,446.676208,1.701109,2.388438,3.165210
432568,46.449802,11.8641,-32.155125,42.305916,46.402546,53.961266,59.906017
432569,50.424248,11.8641,-72.692688,788.745911,0.973941,-1.853863,2.013185


In [12]:
import datetime
from datetime import datetime
temp_arr=x_test[0:,1]
dates=list()
for i in range(0,len(temp_arr)):
    datetime_str=str(int(temp_arr[i]))
    datetime_str=datetime_str[0:6]
    if(datetime_str[4:6]=="00"):
        datetime_str=datetime_str[0:4]+'1'
    datetime_obj = datetime.strptime(datetime_str,"%y%m%d")
    dates.append(str(datetime_obj.date()))


In [56]:
import pandas as pd
df1=pd.DataFrame(y_test,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df1.index.name="DATE"
print("Actual Values:")
df1

Actual Values:


,NO2,O3,NO,CO,PM1,PM2.5,PM10
DATE,,,,,,,
2019-06-04,81.999992,35.592297,-78.999992,705.999939,2.00,2.42,2.820000
2019-09-26,19.000019,49.613506,-7.999984,478.000000,10.17,10.95,12.670000
2019-05-14,58.000042,37.749409,-34.000019,-107.000000,27.40,31.75,41.180000
2019-04-18,82.999977,5.392773,-26.999977,1579.999878,39.41,45.18,50.160000
2019-08-31,5.000026,53.927723,-4.000016,-169.999985,6.90,8.25,9.110000
...,...,...,...,...,...,...,...
2019-03-29,-2.999957,103.541229,63.999969,28.999987,7.13,7.98,8.190000
2019-06-29,76.999962,39.906517,-81.000031,412.999969,1.94,2.61,4.120000
2019-10-02,20.999990,3.235663,-48.000000,-3.000041,52.91,61.52,67.230003


In [57]:
arr=["Y_Actual"]*25
temp_df1=df1.head(25)
temp_df1['Data']=arr
temp_df1.to_excel("xgboost_y_test.xlsx")
temp_df1

/tmp/ipykernel_2698415/2545994415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df1['Data']=arr


,NO2,O3,NO,CO,PM1,PM2.5,PM10,Data
DATE,,,,,,,,
2019-06-04,81.999992,35.592297,-78.999992,705.999939,2.000000,2.420000,2.820000,Y_Actual
2019-09-26,19.000019,49.613506,-7.999984,478.000000,10.170000,10.950000,12.670000,Y_Actual
2019-05-14,58.000042,37.749409,-34.000019,-107.000000,27.400000,31.750000,41.180000,Y_Actual
2019-04-18,82.999977,5.392773,-26.999977,1579.999878,39.410000,45.180000,50.160000,Y_Actual
2019-08-31,5.000026,53.927723,-4.000016,-169.999985,6.900000,8.250000,9.110000,Y_Actual
2019-06-29,9.999954,34.513744,35.000008,51.999977,6.890000,8.320000,10.110000,Y_Actual
2019-07-28,24.000046,6.471327,-37.999985,-68.999977,18.180000,22.040001,23.080000,Y_Actual
2019-07-09,72.000038,11.864100,-93.000031,649.000000,3.120000,4.330000,6.960000,Y_Actual
2019-09-07,52.000031,65.791824,-72.999992,243.999969,20.639999,23.180000,23.709997,Y_Actual


In [58]:
import pandas as pd
df2=pd.DataFrame(y_test_pred,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df2.index.name="DATE"
print("Predicted Values:")
df2

Predicted Values:


,NO2,O3,NO,CO,PM1,PM2.5,PM10
DATE,,,,,,,
2019-06-04,47.849255,11.8641,-80.664482,490.139801,2.968223,3.418961,3.777236
2019-09-26,31.278259,11.8641,-5.546434,341.499512,10.155688,10.203463,13.379510
2019-05-14,51.198467,11.8641,-25.094217,-132.613449,13.112359,15.961463,18.418491
2019-04-18,49.207836,11.8641,-44.744858,681.868164,29.937065,35.181923,34.957455
2019-08-31,33.853252,11.8641,-5.958789,-82.559654,6.641768,8.177238,11.174068
...,...,...,...,...,...,...,...
2019-03-29,34.035519,11.8641,4.500061,-2.882016,6.573976,8.000960,9.871785
2019-06-29,50.424248,11.8641,-70.336044,446.676208,1.701109,2.388438,3.165210
2019-10-02,46.449802,11.8641,-32.155125,42.305916,46.402546,53.961266,59.906017


In [59]:
arr=["Y_Predicted"]*25
temp_df2=df2.head(25)
temp_df2['Data']=arr
temp_df2.to_excel("xgboost_y_test_pred.xlsx")
temp_df2

/tmp/ipykernel_2698415/3694748081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df2['Data']=arr


,NO2,O3,NO,CO,PM1,PM2.5,PM10,Data
DATE,,,,,,,,
2019-06-04,47.849255,11.8641,-80.664482,490.139801,2.968223,3.418961,3.777236,Y_Predicted
2019-09-26,31.278259,11.8641,-5.546434,341.499512,10.155688,10.203463,13.379510,Y_Predicted
2019-05-14,51.198467,11.8641,-25.094217,-132.613449,13.112359,15.961463,18.418491,Y_Predicted
2019-04-18,49.207836,11.8641,-44.744858,681.868164,29.937065,35.181923,34.957455,Y_Predicted
2019-08-31,33.853252,11.8641,-5.958789,-82.559654,6.641768,8.177238,11.174068,Y_Predicted
2019-06-29,33.853252,11.8641,-0.245414,53.225464,5.441239,6.818439,9.191825,Y_Predicted
2019-07-28,46.449802,11.8641,-25.094217,13.388914,20.047733,25.953154,27.592951,Y_Predicted
2019-07-09,53.529594,11.8641,-83.021126,597.863037,2.480992,3.223890,3.449823,Y_Predicted
2019-09-07,44.022236,11.8641,-44.744858,554.178040,20.883434,24.786587,25.654226,Y_Predicted


In [ ]:
#Completed